In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from glob import glob
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def print_transformation(matrix):

    """ 
    """

    # Assuming `matrix` is the 2x3 affine transformation matrix
    a, b, tx = matrix[0]
    c, d, ty = matrix[1]

    # Translation (offset)
    translation_x = tx
    translation_y = ty

    # Rotation angle
    rotation_angle = np.degrees(np.arctan2(c, a))

    # Optional: Scale
    scale_x = np.sqrt(a**2 + c**2)
    scale_y = np.sqrt(b**2 + d**2)

    print(f"[INFO] Translation: ({translation_x}, {translation_y})")
    print(f"[INFO] Rotation Angle: {rotation_angle} degrees")
    print(f"[INFO] Scale: ({scale_x}, {scale_y})")

def align_images(image1, image2, verbose=False):  

    """ 
    """

    # Convert to grayscale
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Feature detection and matching
    sift = cv2.SIFT_create()
    keypoints1, descriptors1 = sift.detectAndCompute(gray1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(gray2, None)

    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)

    # Lowe's ratio test
    good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]

    # Extract matched points
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # Compute the homography matrix
    matrix, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)

    # Get the size of the reference image
    height, width = image1.shape[:2]

    if verbose:
        print_transformation(matrix)

    return(matrix, height, width)

In [6]:
input_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/scratch/jwst_pressreleaseimages/'

input_files1 = glob(f'{input_dir}/jwst_northUp/*.png')
gals = [input_file1.split('/')[-1].split('_')[0] for input_file1 in input_files1]
gals = ['ngc1566']

In [7]:
for gal in gals: 

    print(f'[INFO] Processing: {gal}')

    input_file1 = f'{input_dir}/jwst_northUP/{gal}_northUP.png'
    input_file2 = f'{input_dir}/jwst/{gal}_jwst.png'
    input_file3 = f'{input_dir}/jwsthst/{gal}_jwsthst.png'
    input_file4 = f'{input_dir}/hst/{gal}_hst.png'

    # check if files exist
    if not os.path.isfile(input_file1):
        print(f'[INFO] {input_file1} does not exist')
        continue
    if not os.path.isfile(input_file2):
        print(f'[INFO] {input_file2} does not exist')
        continue
    if not os.path.isfile(input_file3):
        print(f'[INFO] {input_file3} does not exist')
        continue
    if not os.path.isfile(input_file4):
        print(f'[INFO] {input_file4} does not exist')
        continue

    image1 = cv2.imread(input_file1, cv2.IMREAD_COLOR)
    image2 = cv2.imread(input_file2, cv2.IMREAD_COLOR)
    image3 = cv2.imread(input_file3, cv2.IMREAD_COLOR)
    image4 = cv2.imread(input_file4, cv2.IMREAD_COLOR)

    matrix, height, width = align_images(image1, image2)

    # Warp the second image to align with the first
    image2_align = cv2.warpPerspective(image2, matrix, (width, height))
    image3_align = cv2.warpPerspective(image3, matrix, (width, height))
    image4_align = cv2.warpPerspective(image4, matrix, (width, height))

    output_file2 = input_file2.replace('/jwst/', '/jwst_northUp_check/')
    output_file2 = output_file2.replace('_jwst.png', '_jwst_northUp.png')
    output_file3 = input_file3.replace('jwsthst', 'jwsthst_northUp')
    output_file4 = input_file4.replace('hst', 'hst_northUp')

    cv2.imwrite(output_file2, image2_align)
    cv2.imwrite(output_file3, image3_align)
    cv2.imwrite(output_file4, image4_align)

[INFO] Processing: ngc1566


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [5]:
for gal in gals:
    print(f'open jwst_northUp/{gal}_northUp.png jwst_northUp_check/{gal}_jwst_northUp.png')

open jwst_northUp/ngc3627_northUp.png jwst_northUp_check/ngc3627_jwst_northUp.png
open jwst_northUp/ngc1433_northUp.png jwst_northUp_check/ngc1433_jwst_northUp.png
open jwst_northUp/ngc0628_northUp.png jwst_northUp_check/ngc0628_jwst_northUp.png
open jwst_northUp/ngc1385_northUp.png jwst_northUp_check/ngc1385_jwst_northUp.png
open jwst_northUp/ngc1566_northUp.png jwst_northUp_check/ngc1566_jwst_northUp.png
open jwst_northUp/ngc4254_northUp.png jwst_northUp_check/ngc4254_jwst_northUp.png
open jwst_northUp/ngc7496_northUp.png jwst_northUp_check/ngc7496_jwst_northUp.png
open jwst_northUp/ngc5068_northUp.png jwst_northUp_check/ngc5068_jwst_northUp.png
open jwst_northUp/ngc1300_northUp.png jwst_northUp_check/ngc1300_jwst_northUp.png
open jwst_northUp/ngc1365_northUp.png jwst_northUp_check/ngc1365_jwst_northUp.png
open jwst_northUp/ngc2835_northUp.png jwst_northUp_check/ngc2835_jwst_northUp.png
open jwst_northUp/ngc1672_northUp.png jwst_northUp_check/ngc1672_jwst_northUp.png
open jwst_northU